In [9]:
data = """19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
"""
data = open('puzzle.data').read()

import re
from collections import namedtuple
from itertools import combinations, islice

Vector = namedtuple('Vector', 'x y z vx vy vz')

def solve(data: str) -> int:
    vectors = [Vector(*map(int, re.findall(r'[-\d]+', line))) for line in data.splitlines()]

    collisions = 0

    for vector1, vector2 in combinations(vectors, 2):
        det1 = vector1.vx * vector2.vy - vector2.vx * vector1.vy

        if det1 == 0:
            continue

        t1 = ((vector2.x -vector1.x) * vector2.vy - (vector2.y - vector1.y) * vector2.vx) / det1
        
        det2 = vector2.vx * vector1.vy - vector1.vx * vector2.vy
        t2 = ((vector1.x -vector2.x) * vector1.vy - (vector1.y - vector2.y) * vector1.vx) / det2

        x, y = (vector1.x + t1 * vector1.vx, vector1.y + t1 * vector1.vy)
        if t1 >= 0 and t2 >= 0 \
            and 200000000000000 <= x <= 400000000000000 \
            and 200000000000000 <= y <= 400000000000000:
                collisions += 1
    
    return collisions

solve(data)

19523

In [10]:
import z3

def solve2(data: str) -> int:
    vectors = [Vector(*map(int, re.findall(r'[-\d]+', line))) for line in data.splitlines()]
    
    x, y, z, vx, vy, vz, t1, t2, t3 = z3.Reals("x y z vx vy vz t1 t2 t3")
    solver = z3.Solver()

    eq1 = x + t1 * vx == vectors[0].x + t1 * vectors[0].vx
    eq2 = y + t1 * vy == vectors[0].y + t1 * vectors[0].vy
    eq3 = z + t1 * vz == vectors[0].z + t1 * vectors[0].vz
    eq4 = x + t2 * vx == vectors[1].x + t2 * vectors[1].vx
    eq5 = y + t2 * vy == vectors[1].y + t2 * vectors[1].vy
    eq6 = z + t2 * vz == vectors[1].z + t2 * vectors[1].vz
    eq7 = x + t3 * vx == vectors[2].x + t3 * vectors[2].vx
    eq8 = y + t3 * vy == vectors[2].y + t3 * vectors[2].vy
    eq9 = z + t3 * vz == vectors[2].z + t3 * vectors[2].vz

    solver.add(eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9)

    if solver.check() == z3.sat:
        model = solver.model()
        return model[x].as_long() + model[y].as_long() + model[z].as_long()
    
solve2(data)


566373506408017